In [6]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.corpus import stopwords
from string import punctuation
from nltk import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from textblob import Word, TextBlob
# from pattern.en import lemma, lexeme
from nltk.stem import PorterStemmer
cvectorizer = CountVectorizer(ngram_range=(1,1))
vectorizer = TfidfVectorizer(use_idf=True,ngram_range=(1,1))
lem = WordNetLemmatizer()
punct = list(punctuation)
sw = stopwords.words('english')
sp_char = [':','...',"'","''",'``']
# from unidecode import unidecode

In [7]:
df = pd.read_csv('Tweets.csv')

In [51]:
vec_df = pd.DataFrame()
x = 0

In [8]:
df_copy = df.copy()

In [79]:
def processor(text,stem=False,lem_type='wordnet'):
    txt = text.lower()
    txt = re.sub(r'\d+','',txt)
    tkns = word_tokenize(txt)
    tkns = [i for i in tkns if i not in sw and i not in sp_char and i not in punct]
    tkns = [i for i in tkns if i.isalpha()]
    def wordnet_lemmatize(tkns):
        def pos_tag_simplified(tag):
            if tag.startswith('J'):
                return wordnet.ADJ
            elif tag.startswith('V'):
                return wordnet.VERB
            elif tag.startswith('N'):
                return wordnet.NOUN
            elif tag.startswith('R'):
                return wordnet.ADV
            else:
                return None
        tgs = nltk.pos_tag(tkns)
        wtgs = list(map(lambda x: (x[0],pos_tag_simplified(x[1])),tgs))
        lemmatized = []
        for i in wtgs:
            if i[1] != None:
                lemmatized.append(lem.lemmatize(i[0],i[1]))
            else:
                lemmatized.append(lem.lemmatize(i[0]))
        return lemmatized
    def textblob_lemmatize(tkns):
        sent = ' '.join(tkns)
        sent = TextBlob(sent)
        tag_dict = {'J':'a','N':'n','V':'v','R':'r'}
        rds_tags = [(i,tag_dict.get(ii[0],'n')) for i,ii in sent.tags]
        leml = [i.lemmatize(tag) for i,tag in rds_tags]
        return leml
    def pattern_lemmatize(tkns):
        sent = ' '.join(tkns)
        return " ".join([lemma(wd) for wd in sent.split()])
    def stemmer(tkns):
        pr = PorterStemmer()
        stemmed = [pr.stem(i) for i in tkns]
        return ' '.join(stemmed)
    if stem:
        res = stemmer(tkns)
    elif lem_type.lower() == 'wordnet':
        res = wordnet_lemmatize(tkns)
    elif lem_type.lower() == 'pattern':
        res = pattern_lemmatize(tkns)
    elif lem_type.lower() == 'textblob':
        res = textblob_lemmatize(tkns)
    # print(res)
    return ' '.join(res)

In [80]:
def vectorizer(data,stem=False,lem_type='wordnet',vectorizer='tfidf'):
    # f = None
    # global vec_df
    # def check_idf():
        # if f == None:
        #     f = input('Want to use IDF ?(Y/N)')
        #     return True
        # else:
        #     return False
    # flst = list(processor(lst,stem,lem_type))
    # # print(flst)
    # return ' '.join(flst)
    # return flst
    data['clean_text'] = data['text'].apply(lambda x:processor(x,stem=stem,lem_type=lem_type))
    print(data['clean_text'][:5])
    def bow_vectorizer(data):
        bow = cvectorizer.fit_transform(data['clean_text'])
        df_bow = pd.DataFrame(bow.toarray(),columns=cvectorizer.get_feature_names_out())
        return df_bow
    def tfidf_vectorizer(data):
        f = input('Want to use IDF ?(Y/N)')
        if f.lower() == 'y':
            vectorizer = TfidfVectorizer(use_idf=True,ngram_range=(1,1))
            # print('here')
        else:
            vectorizer = TfidfVectorizer(use_idf=False,ngram_range=(1,1))
        tfidf = vectorizer.fit_transform(data['clean_text'])
        df_tfidf = pd.DataFrame(tfidf.toarray(),columns=vectorizer.get_feature_names_out())
        return df_tfidf
    if vectorizer.lower() == 'tfidf':
        res = tfidf_vectorizer(data)
    else:
        res = bow_vectorizer(data)
    return data,res

In [81]:
df,vec_df = vectorizer(df)

0                           virginamerica dhepburn say
1    virginamerica plus added commercial experience...
2    virginamerica today must mean need take anothe...
3    virginamerica really aggressive blast obnoxiou...
4                   virginamerica really big bad thing
Name: clean_text, dtype: object


Want to use IDF ?(Y/N) y


```pyth
df['clean_text'] = df['text'].apply(lambda x:vectorizer(x))

In [82]:
df[['text','clean_text']]

,text,clean_text
0,@VirginAmerica What @dhepburn said.,virginamerica dhepburn say
1,@VirginAmerica plus you've added commercials t...,virginamerica plus added commercial experience...
2,@VirginAmerica I didn't today... Must mean I n...,virginamerica today must mean need take anothe...
3,@VirginAmerica it's really aggressive to blast...,virginamerica really aggressive blast obnoxiou...
4,@VirginAmerica and it's a really big bad thing...,virginamerica really big bad thing
...,...,...
14635,@AmericanAir thank you we got on a different f...,americanair thank get different flight chicago
14636,@AmericanAir leaving over 20 minutes Late Flig...,americanair leave minute late flight warning c...
14637,@AmericanAir Please bring American Airlines to...,americanair please bring american airline blac...
14638,"@AmericanAir you have my money, you change my ...",americanair money change flight answer phone s...


In [83]:
vec_df.head()

,aa,aaaand,aaadvantage,aaalwayslate,aacustomerservice,aadavantage,aadelay,aadv,aadvantage,aafail,...,zip,zipper,zj,zkatcher,zombie,zone,zoom,zrh,zukes,zurich
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
